In [17]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os
import re
from datetime import datetime
from decimal import Decimal, InvalidOperation
import base64
from datetime import datetime, timedelta

pd.options.display.float_format = '{:,.14f}'.format

In [ ]:
# Configurações
load_dotenv()
TOKEN_URL = 'https://api.ticket360.com.br/auth/oauth/access_token'
API_URL = 'https://api.ticket360.com.br'
EVENT_ID = '30642'
MAX_RETRIES = 3  # Número máximo de tentativas
TIMEOUT = 30  # Segundos

def get_access_token():
    """Obtém token com retentativas e timeout ajustável."""
    for attempt in range(MAX_RETRIES):
        try:
            auth_string = f"{os.getenv('CLIENT_ID')}:{os.getenv('CLIENT_SECRET')}"
            auth_base64 = base64.b64encode(auth_string.encode()).decode()
            
            response = requests.post(
                TOKEN_URL,
                headers={
                    "Authorization": f"Basic {auth_base64}",
                    "Content-Type": "application/x-www-form-urlencoded"
                },
                data={"grant_type": "client_credentials"},
                timeout=TIMEOUT
            )
            response.raise_for_status()
            return response.json().get("access_token")
            
        except requests.exceptions.Timeout:
            print(f"Timeout (tentativa {attempt + 1}/{MAX_RETRIES})")
            if attempt < MAX_RETRIES - 1:
                time.sleep(5)  # Espera 5s antes de retentar
                continue
            raise  # Re-lança o erro após últimas tentativas
        except Exception as e:
            print(f"Erro: {type(e).__name__} - {str(e)}")
            return None

def fetch_report(token):
    """Busca relatório com tratamento de erros."""
    try:
        response = requests.get(
            f"{API_URL}/sales/reports/consolidated/{EVENT_ID}?filter=status=paid&ticket.status=active&limit=1000&offset=0",
            headers={"Authorization": f"Bearer {token}"},
            timeout=TIMEOUT
        )
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.Timeout:
        print("Timeout ao buscar relatório")
    except Exception as e:
        print(f"Erro no relatório: {type(e).__name__} - {str(e)}")
    return None



In [18]:
# Execução principal
if __name__ == "__main__":
    try:
        print("Obtendo token...")
        token = get_access_token()
        
        if token:
            print("Buscando relatório...")
            report = fetch_report(token)
            if report:
                print(f"Sucesso! Registros: {len(report.get('sales', []))}")
                print(json.dumps(report, indent=2, ensure_ascii=False))
            else:
                print("Relatório vazio ou não encontrado")
        else:
            print("Falha na autenticação")
    except Exception as e:
        print(f"Erro crítico: {type(e).__name__} - {str(e)}")

Obtendo token...
Buscando relatório...
Sucesso! Registros: 1000
{
  "sales": [
    {
      "id": 9777436,
      "module": "callcenter",
      "date": "2025-04-15T17:14:12-03:00",
      "event.id": 30642,
      "event.name": "Farraial 2025 com Ana Castela, Luan Santana, Simone Mendes, Belo e mais",
      "event.venue.name": "Arena Anhembi",
      "payment.method": "credit-card",
      "payment.method.brand": "0",
      "payment.method.installments": 0,
      "delivery.method": null,
      "delivery.service": null,
      "customer.name": null,
      "customer.document": null,
      "customer.email": null,
      "customer.cellPhoneNumber": null,
      "customer.nationality": null,
      "customer.gender": null,
      "customer.birthDate": null,
      "customer.address.district": null,
      "customer.address.city": null,
      "customer.address.state": null,
      "salestore.type": null,
      "salestore.name": null,
      "salestore.user": null,
      "salestore.address.district": null,


In [ ]:
#Configurando data para coleta do dia anterior
def get_date_range():
    today = datetime.now()
    yesterday = today - timedelta(days=1)
    return yesterday.strftime("%Y-%m-%d")

#Management status
def load_last_offset():
    if os.path.exists("offset.json"):
        with open("offset.json", "r") as f:
            return json.load(f).get("last_offset",0)
    return 0

def save_last_offset(offset):
    with open("offset.json","w") as f:
        json.dump({"last_offset":offset},f)

#Get paginated data with date filter
def fetch_daily_sales(token, target_date):
    all_sales = []
    limit=1000
    offset = load_last_offset()

    while True:
        params = {
            "filter": f"status=paid,date>={target_date}T00:00:00,date<{target_date}T23:59:59",
            "limit": limit,
            "offset": offset
        }

        response = requests.get(
            f"{API_URL}/sales/reports/consolidated/{EVENT_ID}",
            headers={"Authorization": f"Bearer {token}"},
            params=params,
            timeout=30
        )

        data = response.json()

        if not data.get("sales"):
            break

        all_sales.extend(data["sales"])
        offset += limit
        save_last_offset(offset)

        print(f"Página {offset//limit} - {len(all_sales)} registros")

        if len(data["sales"]) < limit:
            break

    save_last_offset(0)  # Reset para próxima execução
    return all_sales

#Daily get data
def save_daily_data(data,date):
    filename = f"sales_{date}.json"
    with open(filename,"w", encoding="utf-8") as f:
        json.dump(data,f,indent=2)
    print(f"Data saved in {filename}")

#Main flow
if __name__ == '__main__':
    try:
        target_date = get_date_range()
        print(f"Collecting date from  {target_date}")

        token = get_access_token()
        if token:
            sales_data = fetch_daily_sales(token, target_date)
            save_daily_data(sales_data,target_date)
    except Exception as e:
        print(f"Erro: {str(e)}")


Página 1 - 1000 registros
Página 2 - 2000 registros
Página 3 - 3000 registros
Página 4 - 4000 registros
Página 5 - 5000 registros
Página 6 - 6000 registros
Página 7 - 7000 registros
Página 8 - 8000 registros
Página 9 - 9000 registros
Página 10 - 10000 registros
Página 11 - 11000 registros
Página 12 - 12000 registros
Página 13 - 13000 registros
Página 14 - 14000 registros
Página 15 - 15000 registros
Página 16 - 16000 registros
Página 17 - 17000 registros
Página 18 - 18000 registros
Página 19 - 19000 registros
Página 20 - 20000 registros
Página 21 - 21000 registros
Página 22 - 22000 registros
Página 23 - 23000 registros
Página 24 - 24000 registros
Página 25 - 25000 registros
Página 26 - 26000 registros
Página 27 - 27000 registros
Página 28 - 28000 registros
Página 29 - 29000 registros
Página 30 - 30000 registros
Página 31 - 31000 registros
Página 32 - 31792 registros
Data saved in sales_2025-08-05.json
